## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
from IPython.display import clear_output



# Import API key
#from config import weather_api_key
from config import g_key
# Configure gmaps API key

gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,-33.65,115.33,64.99,65,97,3.96,light rain
1,1,Wewak,PG,-3.55,143.63,85.23,68,73,6.15,light rain
2,2,Belmonte,BR,-15.86,-38.88,66.20,93,75,8.05,light rain
3,3,Mataura,NZ,-46.19,168.86,55.99,68,25,10.00,scattered clouds
4,4,Tambopata,PE,-12.73,-69.18,69.67,60,32,2.66,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 12
What is the maximum temperature you would like for your trip? 100


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
city_temp_data_df = city_data_df.loc[(city_data_df["Max Temp"]<= max_temp)& \
                                     (city_data_df["Max Temp"]>= min_temp)]
city_temp_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,-33.65,115.33,64.99,65,97,3.96,light rain
1,1,Wewak,PG,-3.55,143.63,85.23,68,73,6.15,light rain
2,2,Belmonte,BR,-15.86,-38.88,66.20,93,75,8.05,light rain
3,3,Mataura,NZ,-46.19,168.86,55.99,68,25,10.00,scattered clouds
4,4,Tambopata,PE,-12.73,-69.18,69.67,60,32,2.66,scattered clouds
5,5,Aksu,CN,41.12,80.26,51.85,39,1,1.32,clear sky
6,6,Victoria,HK,22.29,114.16,84.99,55,86,1.99,overcast clouds
7,7,Vaini,TO,-21.20,-175.20,78.80,83,40,10.29,smoke
8,8,Saquarema,BR,-22.92,-42.51,64.74,77,16,10.40,few clouds
9,9,Kapaa,US,22.08,-159.32,73.99,100,90,9.17,light rain


In [5]:
# 4a. Determine if there are any empty rows.
city_temp_data_df.count()


City_ID                668
City                   668
Country                663
Lat                    668
Lng                    668
Max Temp               668
Humidity               668
Cloudiness             668
Wind Speed             668
Current Description    668
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
city_temp_data_df= city_temp_data_df.dropna()
city_temp_data_df.count()


City_ID                663
City                   663
Country                663
Lat                    663
Lng                    663
Max Temp               663
Humidity               663
Cloudiness             663
Wind Speed             663
Current Description    663
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = city_temp_data_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Busselton,AU,64.99,light rain,-33.65,115.33,
1,Wewak,PG,85.23,light rain,-3.55,143.63,
2,Belmonte,BR,66.20,light rain,-15.86,-38.88,
3,Mataura,NZ,55.99,scattered clouds,-46.19,168.86,
4,Tambopata,PE,69.67,scattered clouds,-12.73,-69.18,
5,Aksu,CN,51.85,clear sky,41.12,80.26,
6,Victoria,HK,84.99,overcast clouds,22.29,114.16,
7,Vaini,TO,78.80,smoke,-21.20,-175.20,
8,Saquarema,BR,64.74,few clouds,-22.92,-42.51,
9,Kapaa,US,73.99,light rain,22.08,-159.32,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
counter=0
index_2=0
# 6b. Iterate through the hotel DataFrame.
for index,row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    #Add lat and lng to params dictionary
    params["location"] = f"{lat},{lng}"
    # 6e. Make request and retrieve the JSON data from the search.
    response=requests.get(base_url,params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
        print(f'Success {index_2} of {len(hotel_df)} | {"{:.2f}".format((index_2/len(hotel_df)*100))}%')
        counter+=1
        index_2+=1
        if counter == 10:
            clear_output()
            counter=0
    except IndexError:
        print(f'\nFaliure {index_2} of {len(hotel_df)} | {"{:.2f}".format((index_2/len(hotel_df.index)*100))}% \n\t\tSkipping...\n')
        counter+=1
        index_2+=1
        if counter == 10:
            clear_output()
            counter=0
        pass
print("-"*30+"\nCompleted\t100.00%\n"+"-"*30)

Success 660 of 663 | 99.55%
Success 661 of 663 | 99.70%
Success 662 of 663 | 99.85%
------------------------------
Completed	100.00%
------------------------------


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_df = hotel_df.dropna()
clean_df.count()

City                   663
Country                663
Max Temp               663
Current Description    663
Lat                    663
Lng                    663
Hotel Name             663
dtype: int64

In [10]:
# 8a. Create the output File (CSV)
output_data_file = "Weatherpy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F<dd>
</dl> 
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 

locations = clean_df[["Lat","Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# 11b. Display the figure



fig

Figure(layout=FigureLayout(height='420px'))